In [1]:
# import packages
import numpy as np
# import scipy.integrate as integrate
# from sympy import *
import time

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation

from matplotlib import cm
from matplotlib.ticker import LinearLocator
## set default plot size
plt.rcParams["figure.figsize"] = (10, 7)
## set global font size
plt.rcParams.update({'font.size':14})

# import astropy.units as u
from astropy import constants as const

from scipy.interpolate import interp1d


# Leapfrogging Particle Trajectories

This notebook focuses on computing and animating the trajectories of individual particles within our two-hammer potential rather than the full solution $\delta f$. I will do this in two ways:
1. By using the analytical solution to Hamilton's equations, assuming two delta-function perturbations.
2. By numerically solving Hamilton's equations, which won't necessarily assume delta-function perturbations.